In [2]:
import pandas as pd
import altair as alt

In [11]:
data = pd.DataFrame({
    'x': list(range(1, 11)) * 2,          # eixo X (tempo ou índice)
    'y': [i*2 for i in range(1, 11)] +    # valores da série A
          [i*3 for i in range(1, 11)],    # valores da série B
    'serie': ['A']*10 + ['B']*10          # identificador da série
})

# Gráfico de linhas com múltiplas séries
chart = alt.Chart(data).mark_line().encode(
    x='x',
    y='y',
    color='serie'   # diferencia as séries pela cor
).properties(
    title='Gráfico de linhas com múltiplas séries'
)

chart.show()


alt.Chart(...)

In [46]:
raw_sales = pd.read_csv("Datasets/video games sales.csv")
raw_imdb = pd.read_csv("Datasets/imdb-videogames.csv")

df_sales = raw_sales[["Name", "Year", "Genre", "Global_Sales"]].dropna()
df_imdb = raw_imdb[["name", "year", "rating", "Action", "Adventure", "Comedy", "Crime", "Family", "Mystery", "Sci-Fi", "Thriller"]].dropna()

df_sales_year = df_sales.groupby("Year")[["Global_Sales"]].mean(numeric_only=True).reset_index()

df_imdb.rename(columns={"year": "Year", "name": "Name"}, inplace=True)
df_quant_gen = df_imdb.groupby("Year")[["rating", "Action", "Adventure", "Comedy", "Crime", "Family", "Mystery", "Sci-Fi", "Thriller"]].sum().reset_index()

merged_bar = pd.merge(df_sales, df_imdb, on=["Name", "Year"], how="inner")
merged_line = pd.merge(df_sales_year, df_quant_gen, on=["Year"], how="inner")

df_ratings = merged_bar.groupby("Genre")["rating"].mean().reset_index()

genres = ['Action', 'Adventure', 'Comedy', 'Crime', 'Family', 'Mystery', 'Sci-Fi', 'Thriller']

df_count = merged_line.melt(
    id_vars=['Year'],
    value_vars=genres,
    var_name='Genre',
    value_name='Value'
)
df_count['Metric'] = 'Count'

df_sales = pd.DataFrame({
    'Year': merged_line['Year'].repeat(len(genres)),
    'Genre': genres * len(merged_line),
    'Value': merged_line['Global_Sales'].repeat(len(genres)),
    'Metric': 'Sales'
})

dfs = []
for g in genres:
    df_temp = pd.DataFrame({
        'Year': merged_line['Year'],
        'Genre': g,
        'Value': merged_line['Global_Sales'] / merged_line[g],
        'Metric': 'Avg_per_game'
    })
    dfs.append(df_temp)

df_avg = pd.concat(dfs, ignore_index=True)


# Criar lista de DataFrames, um para cada gênero
# dfs = []
# for g in genres:
#     df_temp = pd.DataFrame({
#         'x': merged_line['Year'],
#         'y': merged_line['Global_Sales'] / merged_line[g],
#         'Gênero': g
#     })
#     dfs.append(df_temp)

# # Concatenar tudo
# df_line = pd.concat(dfs, ignore_index=True)

# for g in genres:
#     df_line = pd.DataFrame({
#         'Year': merged_line['Year'],
#         'Metric': 'Avg_per_game',
#         #'Quant': merged_line[g],
#         #'Sales': merged_line['Global_Sales'] ,
#         'Sales/Quant': merged_line['Global_Sales'] / merged_line[g]
#     })

df_long = pd.concat([df_count, df_sales, df_avg], ignore_index=True)



# df_line = pd.DataFrame({
#     'x': merged_line["Year"].repeat(8),
#     'y': [pd.to_numeric(merged_line["Global_Sales"]) / pd.to_numeric(merged_line["Action"]),
#             pd.to_numeric(merged_line["Global_Sales"]) / pd.to_numeric(merged_line["Adventure"]),
#             pd.to_numeric(merged_line["Global_Sales"]) / pd.to_numeric(merged_line["Comedy"]),
#             pd.to_numeric(merged_line["Global_Sales"]) / pd.to_numeric(merged_line["Crime"]),
#             pd.to_numeric(merged_line["Global_Sales"]) / pd.to_numeric(merged_line["Family"]),
#             pd.to_numeric(merged_line["Global_Sales"]) / pd.to_numeric(merged_line["Mystery"]),
#             pd.to_numeric(merged_line["Global_Sales"]) / pd.to_numeric(merged_line["Sci-Fi"]),
#             pd.to_numeric(merged_line["Global_Sales"]) / pd.to_numeric(merged_line["Thriller"])],
#     'serie' : ['Action', 'Adventure', 'Comedy', 'Crime', 'Family', 'Mystery', 'Sci-Fi', 'Thriller'] * len(merged_line)
# })



In [52]:
year_min = int(df_count['Year'].min())
year_max = int(df_count['Year'].max())
year_param = alt.param(name='Ano', bind=alt.binding_range(min=year_min, max=year_max, step=1), value=2000)

type_dropdown = alt.param(
    name='Metrica Escolhida',
    bind=alt.binding_select(options=df_long['Metric'].unique().tolist()),
    value='Avg_per_game'   # valor inicial
)

graph_rating = alt.Chart(df_ratings).mark_bar().encode(
    x=alt.X("Genre:O", title="Genre"),
    y=alt.Y("rating:Q", title="Average Rating"),
    color=alt.Color("Genre:N", legend=None)
).properties(
    width=500,
    title = "Nota média por gênero de jogo"
)

graph_sales = alt.Chart(df_long).mark_line(point=True).encode(
    x=alt.X('Year:O', title='Ano'),
    y=alt.Y('Value:Q', title='Vendas globais (M) / quantidade de jogos'),
).properties(
    title='Relação entre Vendas Globais e Quantidade de Jogos por Gênero ao Longo dos Anos'
).transform_filter(
    alt.datum.Year >= year_param,
    alt.datum.Metric == type_dropdown
).add_params(year_param, type_dropdown).properties(width=600, height=350)

In [ ]:
graph_rating

alt.Chart(...)

In [53]:
graph_sales.interactive()

alt.Chart(...)